## ORDINAL REGRESSION PROBLEM

In [1]:
# CARGAMOS LIBRERIAS
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os
import math

NUESTRO PROBLEMA A RESOLVER ES EL DE ESTIMAR UN RANKING DE ACTIVOS EN BASE A UN CONJUNTO DE MÉTRICAS. UN RANKING ES UN CONJUNTO DE VALORES ORDENADO DEPENDIENDO A SU 
RELEVANCIA CON RESPECTO A UN CIERTO CRITERIO. POR LO TANTO, NUESTRA TAREA DE APRENDIZAJE ES LA DE ESTIMAR UN CONJUNTO ORDENADO DE ACTIVOS.

ESTA TAREA SE CONOCE COMO REGRESION ORDINAL. ES UN TIPO DE MODELO DE CLASIFICACIÓN EN EL CUAL SE BUSCA ENCONTRAR LA DIRECCIÓN DEL ORDEN DE CADA ELEMENTO DENTRO DE NUESTRO
CONJUNTO DE DATOS.

PARA PODER ESTIMAR UN RANKING DEBEMOS POR LO TANTO REALIZAR UNA TAREA DE CLASIFICACIÓN DE CADA ACTIVO. EN ESTE CASO SE DEBERÁ PREPROCESAR NUESTRAS ETIQUETAS DE MANERA TAL
QUE EL MODELO PUEDA ESTIMAR LA POSICIÓN Y EL ORDEN CORRECTO DE CADA ACTIVO. POR LO TANTO, EL PREPROCESAMIENTO DEBE REALIZARSE DE LA SIGUIENTE MANERA:
- [0] -> [0, 0, 0, 0]
- [1] -> [1, 0, 0, 0]
- [2] -> [1, 1, 0, 0]  
- [3] -> [1, 1, 1, 0]
- [4] -> [1, 1, 1, 1]

EN VEZ DE UTILIZAR UNA FUNCIÓN SOFTMAX Y CODIFICAR COMO ONE HOT, UTILIZAREMOS UNA FUNCIÓN SIGMOIDEA Y LA CODIFICACIÓN DETALLADA ANTERIORMENTE

In [38]:
ACTIVOS = 97
len_array = 2000
ranking = np.ndarray((len_array, ACTIVOS))
ranking.shape

(2000, 97)

In [50]:
ranking[:] = range(ACTIVOS)
ranking

array([[ 0.,  1.,  2., ..., 94., 95., 96.],
       [ 0.,  1.,  2., ..., 94., 95., 96.],
       [ 0.,  1.,  2., ..., 94., 95., 96.],
       ...,
       [ 0.,  1.,  2., ..., 94., 95., 96.],
       [ 0.,  1.,  2., ..., 94., 95., 96.],
       [ 0.,  1.,  2., ..., 94., 95., 96.]])

In [58]:
def shuffle_along_axis(a, axis):
    idx = np.random.rand(*a.shape).argsort(axis=axis)
    return np.take_along_axis(a,idx,axis=axis)

In [64]:
ranking = shuffle_along_axis(ranking,1)
ranking

array([[ 5.,  9.,  1., ..., 38., 13., 89.],
       [76., 15., 28., ..., 84., 83., 74.],
       [66., 75., 63., ..., 85., 84., 23.],
       ...,
       [54., 46., 41., ..., 12., 18., 29.],
       [34., 69., 77., ..., 65., 81.,  5.],
       [29., 72., 27., ..., 59., 30.,  5.]])

In [84]:
encoded_ranking = np.zeros((ranking.shape[0],ranking.shape[1],ranking.shape[1]))
encoded_ranking

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [97]:
for i in range(ranking.shape[0]):
    print(ranking[i,i])
    # encoded_ranking[:,:,:int(ranking[i,i])] = 1

5.0
15.0
63.0
51.0
15.0
66.0
56.0
54.0
77.0
5.0
21.0
58.0
43.0
24.0
91.0
31.0
16.0
91.0
16.0
17.0
94.0
78.0
68.0
79.0
47.0
53.0
53.0
79.0
18.0
70.0
5.0
92.0
34.0
4.0
45.0
67.0
38.0
15.0
89.0
91.0
43.0
51.0
63.0
70.0
56.0
55.0
7.0
21.0
24.0
11.0
43.0
12.0
69.0
42.0
29.0
11.0
38.0
8.0
20.0
48.0
41.0
94.0
45.0
81.0
93.0
15.0
94.0
74.0
57.0
86.0
17.0
44.0
20.0
0.0
15.0
59.0
30.0
11.0
90.0
68.0
38.0
30.0
51.0
9.0
50.0
50.0
10.0
73.0
76.0
77.0
14.0
74.0
55.0
82.0
5.0
73.0
31.0


IndexError: index 97 is out of bounds for axis 1 with size 97

In [80]:
for l in range(encoded_ranking.shape[0]):
    for i in range(encoded_ranking.shape[1]):
        encoded_ranking[l,i,:int(ranking[i,i])] = 1
encoded_ranking

array([[[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]],

       [[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.]],

       ...,

       [[1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        ...,
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0., 0.],
        [1., 1., 1., ..., 0., 0.